In [1]:
import pandas as pd
#!pip install geopy

In [5]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()
#location = geolocator.geocode("Julich, Germany")

In [6]:
def getCC(i):
    return "{}, {}".format(df.loc[i, 'City'], df.loc[i, 'Country'])

def getLoc(cc):
    location = geolocator.geocode(cc)
    return "{}, {}".format(location.latitude, location.longitude)

In [7]:
df = pd.read_excel("/Users/akeshavan/Downloads/5.24.18_questions.xlsx")#"/Users/akeshavan/Downloads/4.9.18.xls")
df[['City','Country']]
df['city_country'] = df.index.map(getCC)
#df.head()

In [12]:
for i, row in df.iterrows():
    try:
        ll = getLoc(row.city_country)
        df.loc[i, 'latlong'] = ll
    except AttributeError:
        print(row.City, row.Country, i)
    except OSError:
        print("os error", row.City, row.Country, i)

MontrÃ©al Canada 0
JÃ¼lich Germany 3
JÃ¼lich, Nordrhein-Westfahlen Germany 24
JÃ¼lich Germany 29
LinkÃ¶ping Sweden 48
Sart Tilman, LiÃ¨ge Belgium 89


In [13]:
def cleanup(idx, city, country):
    df.loc[idx, 'city'] = 'Julich'
    df.loc[idx, 'country'] = 'Germany'
    df['city_country'] = df.index.map(getCC)
    df.loc[idx, 'latlong'] = getLoc('{}, {}'.format(city, country))

In [14]:
cleanup(0, "Montreal", "Canada")
cleanup(3, "Julich", "Germany")
cleanup(24, "Julich", "Germany")
cleanup(29, "Julich", "Germany")
cleanup(48, "Linkoping", "Sweden")
cleanup(89, "Liege", "Belgium")

for i, row in df.iterrows():
    try:
        ll = getLoc(row.city_country)
        df.loc[i, 'latlong'] = ll
    except AttributeError:
        print(row.city, row.country, i)

In [15]:
df2 = pd.DataFrame(df.groupby('latlong').apply(lambda x: x.count()['latlong']))

In [16]:
df2['lat'] = df2.index.map(lambda x: x.split(',')[0])
df2['long'] = df2.index.map(lambda x: x.split(',')[1])
df2.rename(columns= {0: "count"}, inplace=True)
df2.head()

,count,lat,long
latlong,,,
"-22.9110137, -43.2093727",1,-22.9110137,-43.2093727
"-33.7510792, 150.6941684",1,-33.7510792,150.6941684
"-33.8548157, 151.2164539",1,-33.8548157,151.2164539
"-33.928992, 18.417396",1,-33.928992,18.417396
"-37.7382431, 145.0433922",1,-37.7382431,145.0433922


In [17]:
output = []
for i, row in df2.iterrows():
    output += [float(row.lat), float(row.long), float(row['count'])/max(df2['count'])]

In [18]:
from nipype.utils.filemanip import save_json

In [19]:
save_json("./globe/populationBrainHack.json", [["2018", output]])

In [20]:
save_json("../hackathon2018/globe/populationBrainHack.json", [["2018", output]])